In [13]:
from ibm_watsonx_ai import APIClient
from langchain_ibm import ChatWatsonx

project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd"
url="https://us-south.ml.cloud.ibm.com"
apikey="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_"

parameters = {
    "frequency_penalty": 0,
    "max_tokens": 20,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

def create_chat_model(model_id):
    return ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=apikey,
    project_id=project_id,
    params=parameters,
)
model_id = "ibm/granite-3-8b-instruct"
chat_model = create_chat_model(model_id)
chat_model.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 60, 'total_tokens': 70}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-19306961b52f6c4a587de81a6e9bab79', usage_metadata={'input_tokens': 60, 'output_tokens': 10, 'total_tokens': 70})

In [1]:
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool


In [ ]:
class State(TypedDict):
    #{"messages":["your message"]}
    messages: Annotated[list, add_messages]



def chatbot(state:State):
    response = chat_model.invoke(state['messages'])
    return {"messages":[response]}


graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START,"chatbot")
graph_builder.add_edge("chatbot",END)
graph = graph_builder.compile()

In [16]:
graph.invoke({"messages":['who are you?']})

{'messages': [HumanMessage(content='who are you?', additional_kwargs={}, response_metadata={}, id='d4855a97-7e0d-4be6-94f0-e79e2a1bf82d'),
  AIMessage(content='I am Granite, an AI developed by IBM Research—a research organization spread across the glo', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 63, 'total_tokens': 83}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'length'}, id='chatcmpl-301f426318e6e3900774074380bd53ab', usage_metadata={'input_tokens': 63, 'output_tokens': 20, 'total_tokens': 83})]}

In [18]:
graph.invoke({"messages":["Hi","This is Mohammad"]})

{'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='0e723c37-f76d-4958-ac68-830c2518b08e'),
  HumanMessage(content='This is Mohammad', additional_kwargs={}, response_metadata={}, id='a46bbea9-3562-4348-9fd7-40e0f2434e6c'),
  AIMessage(content="Hello Mohammad, it's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 71, 'total_tokens': 91}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'length'}, id='chatcmpl-f41d9e932dc71daa384a383a0517183c', usage_metadata={'input_tokens': 71, 'output_tokens': 20, 'total_tokens': 91})]}

In [ ]:
while True:
    user_input = input ("You: ")
    if user_input in ['q','quit','exit']:
        print("Bye!")
        break
    response = graph.invoke({"messages":[user_input]})
    print("Assistant: ", response["messages"][-1].content)

Assistant:  Hello! How can I assist you today? I'm here to provide information and answer your questions to
Assistant:  2 + 2 equals 5.


KeyboardInterrupt: 

In [23]:
class user(TypedDict):
    name:str
    age : int
    email:str

data = {"name":"ali", "age":20, "email" : "a@a.com"}
user(data)

data = {"name":"ali", "age":'20', "email" : "a@a.com"}
user(data)

{'name': 'ali', 'age': '20', 'email': 'a@a.com'}

In [31]:
from pydantic import BaseModel, Field, EmailStr
from typing import Annotated, TypedDict, Optional, Literal

In [36]:
class user(BaseModel):
    name:str =      Field(...,min_length = 3, max_length = 50, description = "name of the user")
    age : int =      Field(...,gt = 0,  description = "age of the user")
    email:EmailStr = Field(...,  description = "email of the user")
    sex: Optional[Literal["male","female","other"]]="other"

user (name="ali", age=20, email = "a@a.com")

user(name='ali', age=20, email='a@a.com', sex='other')

In [42]:
user (name="ali", age='2', email = "a@a.com", sex = 'male')

user(name='ali', age=2, email='a@a.com', sex='male')

In [43]:
from typing import Annotated, get_origin, get_args, get_type_hints

In [57]:
def square( x: Annotated[int, (0,100)])  -> int:
    type_hints = get_type_hints(square, include_extras = True)

    hint = type_hints['x']
    if get_origin(hint) is Annotated:
        hint_type, *hint_args = get_args(hint)
        low, high = hint_args[0]
        if not (low <= x <= high):
          raise ValueError(f"input must be in {low} {high} ")
        
    return x*x 


print(square(10))    

100


In [59]:
print(square(150))   

ValueError: input must be in 0 100 